In [34]:
import tensorflow as tf
from tensorflow.keras import layers, models

# 절반씩 줄여보기도 하기

def create_model(input_shape, num_classes):
    model = models.Sequential()

    # Convolution 1
    model.add(layers.Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.BatchNormalization())

    # Maxpooling 1
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # Convolution 2
    model.add(layers.Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(layers.BatchNormalization())

    # Maxpooling 2
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # Convolution 3
    model.add(layers.Conv2D(256, kernel_size=(3, 3), activation='relu'))
    model.add(layers.BatchNormalization())

    # Maxpooling 3
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # Flatten
    model.add(layers.Flatten())

    # FC 1
    model.add(layers.Dense(2048, activation='relu'))
    model.add(layers.Dropout(0.5))

    # FC 2
    model.add(layers.Dense(2048, activation='relu')) 
    model.add(layers.Dropout(0.5))

    # FC 3 (출력층)
    model.add(layers.Dense(num_classes, activation='softmax'))

    # 모델 컴파일
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', 'AUC'])
    
    return model

In [36]:
def load_data_and_labels(data_folder, num_classes, img_size=(28, 28), color_mode='grayscale'):
    X = []
    y_onehot = []
    
    for idx, class_folder in enumerate(sorted(os.listdir(data_folder))):
        class_path = os.path.join(data_folder, class_folder)
        
        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            img = load_img(img_path, target_size=img_size, color_mode=color_mode)
            X.append(img_to_array(img))
            
            onehot = np.zeros(num_classes)
            onehot[idx] = 1
            y_onehot.append(onehot)
    
    return np.array(X) / 255.0, np.array(y_onehot)

In [39]:
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# 데이터 경로
base_path = r'C:\Users\pc\Desktop\CNN'
train_data_folder = os.path.join(base_path, 'Multiclass_All_Train_Image')
val_data_folder = os.path.join(base_path, 'Multiclass_All_Validation_Image')
test_data_folder = os.path.join(base_path, 'Multiclass_All_Test_Image')

num_classes = 10

# 데이터 로드
X_train, y_train_onehot = load_data_and_labels(train_data_folder, num_classes)
X_val, y_val_onehot = load_data_and_labels(val_data_folder, num_classes)
X_test, y_test_onehot = load_data_and_labels(test_data_folder, num_classes)

# 모델 생성
input_shape = X_train[0].shape
model = create_model(input_shape, num_classes)

# 모델 학습
epochs = 50
history = model.fit(X_train, y_train_onehot, validation_data=(X_val, y_val_onehot), epochs=epochs)

# 모델 평가
test_loss, test_accuracy, test_auc = model.evaluate(X_test, y_test_onehot)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)
print("Test AUC:", test_auc)

Epoch 1/50
1250/1250 [==============================] - 9s 7ms/step - loss: 2.2506 - accuracy: 0.1504 - auc: 0.5785 - val_loss: 2.4264 - val_accuracy: 0.1692 - val_auc: 0.5977
Epoch 2/50
1250/1250 [==============================] - 8s 6ms/step - loss: 2.1894 - accuracy: 0.1613 - auc: 0.6054 - val_loss: 4.5583 - val_accuracy: 0.1508 - val_auc: 0.5760
Epoch 3/50
1250/1250 [==============================] - 8s 6ms/step - loss: 2.1668 - accuracy: 0.1700 - auc: 0.6216 - val_loss: 3.8672 - val_accuracy: 0.1418 - val_auc: 0.5598
Epoch 4/50
1250/1250 [==============================] - 8s 6ms/step - loss: 2.1481 - accuracy: 0.1771 - auc: 0.6353 - val_loss: 3.2318 - val_accuracy: 0.1678 - val_auc: 0.5886
Epoch 5/50
1250/1250 [==============================] - 8s 6ms/step - loss: 2.1302 - accuracy: 0.1852 - auc: 0.6477 - val_loss: 7.3223 - val_accuracy: 0.1338 - val_auc: 0.5434
Epoch 6/50
1250/1250 [==============================] - 8s 6ms/step - loss: 2.1155 - accuracy: 0.1926 - auc: 0.6570 - va

1250/1250 [==============================] - 8s 6ms/step - loss: 1.9634 - accuracy: 0.2405 - auc: 0.7242 - val_loss: 13.5672 - val_accuracy: 0.2044 - val_auc: 0.6204
Epoch 48/50
1250/1250 [==============================] - 8s 6ms/step - loss: 1.9719 - accuracy: 0.2368 - auc: 0.7232 - val_loss: 5.9728 - val_accuracy: 0.2120 - val_auc: 0.6428
Epoch 49/50
1250/1250 [==============================] - 8s 6ms/step - loss: 1.9670 - accuracy: 0.2388 - auc: 0.7254 - val_loss: 8.0301 - val_accuracy: 0.2082 - val_auc: 0.6244
Epoch 50/50
157/157 [==============================] - 1s 3ms/step - loss: 5.2146 - accuracy: 0.2020 - auc: 0.6317
Test Loss: 5.214554786682129
Test Accuracy: 0.20200000703334808
Test AUC: 0.6317453980445862


In [40]:
from sklearn.metrics import confusion_matrix
import numpy as np

# 모델을 사용하여 테스트 데이터에 대한 예측을 수행합니다.
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)  # 다중 클래스 분류 문제에서는 가장 높은 확률을 가진 클래스를 선택합니다.

# 실제 레이블을 가져옵니다.
y_true = np.argmax(y_test_onehot, axis=1)

# Confusion Matrix를 생성합니다.
conf_matrix = confusion_matrix(y_true, y_pred_classes)

# 정확도를 계산합니다.
accuracy = np.trace(conf_matrix) / float(np.sum(conf_matrix))

print("Confusion Matrix:")
print(conf_matrix)
print("Accuracy:", accuracy)

157/157 [==============================] - 0s 1ms/step
Confusion Matrix:
[[ 65  57   9 110  40  53  30  68  37  31]
 [ 25  63  15 111  54  52  26  76  31  47]
 [ 37  76  25  98  32  51  37  58  48  38]
 [ 29  61  11 116  44  59  18  85  31  46]
 [  3   3   0  70 302  16   5  78  11  12]
 [ 48  55   8 106  33  90  37  59  28  36]
 [ 15  22   0 118  42  27  67  80  61  68]
 [ 28  16   2 110  30  40  45 128  47  54]
 [ 23  24   0 106  37  28  73  83  48  78]
 [ 17  16   0 102  41  23  63  73  59 106]]
Accuracy: 0.202
